In [1]:
# Using Object detection feature of Custom Vision Services 
# In this demo we'll use json output of VoTT Tool 
# Before starting you need to install customvision api using pip install

In [2]:
#Create Custom Vision Service Project
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region

# My Custom Vision AI Keys
training_key = "YOUR TRAINING KEY"
prediction_key = "YOUR PREDICTION KEY"

trainer = training_api.TrainingApi(training_key)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection")

# Create a new project
project = trainer.create_project("Office", domain_id=obj_detection_domain.id)
print ("Project Created...")

Project Created...


In [3]:
import json
import os
import sys

#FileName of your folder and jsonfile created by VoTT
FileName = "Office"

#import the json file
json_file = FileName + ".json"
with open(json_file,'r',encoding="utf-8") as f:
    data = json.load(f)

#import all tags in json file
inputTags = data["inputTags"].split(",")
print(inputTags)

['Çay', 'TürkKahvesi']


In [4]:
#Create all Tags
for tag in inputTags:
    trainer.create_tag(project.id, tag)
    print(tag)

Çay
TürkKahvesi


In [5]:
imgtags = {}

#Get all tagsIDs created on custom vision and add into a dictionary
for tag in trainer.get_tags(project.id):
    imgtags[tag.name] = tag.id

print(imgtags)

{'Çay': '36c60fe0-8d03-4d3f-b8e9-3d66f2b55078', 'TürkKahvesi': '17065443-6d27-4b26-b8ee-8b05aac3b3b5'}


In [6]:
#Help to sort alphanumeric
import re

def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [7]:
imgtagdic = {}

# Group all the photos by tags using their ROIs(Region of Images)
# image pixel normalization for Custom Vision
# tleft = x1/w 
# ttop = y1/h
# twidth = (x2-x1)/w
# theight = (y2-y1)/h
for a in data["visitedFrames"]:
    print(a)
    try:
        for key in data["frames"][str(a)]:
            print(key)
            x1 = key["x1"]
            y1 = key["y1"]
            x2 = key["x2"]
            y2 = key["y2"]        
            h = key["height"]
            w = key["width"]
            tleft = x1/w 
            ttop = y1/h
            twidth = (x2-x1)/w
            theight = (y2-y1)/h
            try:
                tag = key["tags"][0]
                # Defining UniqueID per tags in photo below  
                imgtagdic[str(a)+','+ str(key["name"])+ tag] = [imgtags[tag],tleft,ttop,twidth,theight]
            except:
                print(str(a)+ tag +"error")
    except:
        print("Error Occured")

#Print all 
print(imgtagdic)

0
{'x2': 547, 'y2': 404, 'width': 789, 'id': 0, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 94, 'type': 'Rectangle', 'y1': 78}
1
{'x2': 564, 'y2': 478, 'width': 789, 'id': 1, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 87, 'type': 'Rectangle', 'y1': 131}
2
{'x2': 550, 'y2': 488, 'width': 789, 'id': 2, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 87, 'type': 'Rectangle', 'y1': 139}
3
{'x2': 553, 'y2': 498, 'width': 789, 'id': 3, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 68, 'type': 'Rectangle', 'y1': 123}
4
{'x2': 564, 'y2': 408, 'width': 789, 'id': 4, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 162, 'type': 'Rectangle', 'y1': 143}
5
{'x2': 517, 'y2': 491, 'width': 789, 'id': 5, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 74, 'type': 'Rectangle', 'y1': 178}
6
{'x2': 515, 'y2': 440, 'width': 789, 'id': 6, 'height': 592, 'tags': ['Çay'], 'name': 1, 'x1': 124, 'type': 'Rectangle', 'y1': 163}
7
{'x2': 455, 'y2': 388, 'width': 789, 'id': 7, 'height': 592, 'tags

In [8]:
len(imgtagdic)

46

In [9]:
imgtagdic

{'0,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.11913814955640051,
  0.13175675675675674,
  0.5741444866920152,
  0.5506756756756757],
 '1,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.11026615969581749,
  0.22128378378378377,
  0.6045627376425855,
  0.5861486486486487],
 '10,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.21166032953105196,
  0.2922297297297297,
  0.4740177439797212,
  0.4239864864864865],
 '11,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.2256020278833967,
  0.27533783783783783,
  0.46514575411913817,
  0.41047297297297297],
 '12,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.26878238341968913,
  0.30569948186528495,
  0.422279792746114,
  0.37132987910189985],
 '13,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.08365019011406843,
  0.23986486486486486,
  0.5627376425855514,
  0.5202702702702703],
 '14,1Çay': ['36c60fe0-8d03-4d3f-b8e9-3d66f2b55078',
  0.1596958174904943,
  0.2635135135135135,
  0.5868187579214195,
  0.533783783783783

In [10]:
import os

allphotos = sorted_aphanumeric(os.listdir(FileName+ "/"))

# Go through the data table above and create the images
print ("images are ready")

images are ready


In [11]:
print(len(allphotos))

46


In [12]:
print(allphotos)

['IMG_0758.JPG', 'IMG_0759.JPG', 'IMG_0760.JPG', 'IMG_0761.JPG', 'IMG_0762.JPG', 'IMG_0763.JPG', 'IMG_0764.JPG', 'IMG_0765.JPG', 'IMG_0766.JPG', 'IMG_0767.JPG', 'IMG_0768.JPG', 'IMG_0769.JPG', 'IMG_0770.JPG', 'IMG_0771.JPG', 'IMG_0772.JPG', 'IMG_0773.JPG', 'IMG_0774.JPG', 'IMG_0775.JPG', 'IMG_0776.JPG', 'IMG_0777.JPG', 'IMG_0778.JPG', 'IMG_0779.JPG', 'IMG_0780.JPG', 'IMG_0781.JPG', 'IMG_0782.JPG', 'IMG_0784.JPG', 'IMG_0785.JPG', 'IMG_0786.JPG', 'IMG_0787.JPG', 'IMG_0788.JPG', 'IMG_0789.JPG', 'IMG_0790.JPG', 'IMG_0791.JPG', 'IMG_0792.JPG', 'IMG_0793.JPG', 'IMG_0794.JPG', 'IMG_0795.JPG', 'IMG_0796.JPG', 'IMG_0797.JPG', 'IMG_0798.JPG', 'IMG_0799.JPG', 'IMG_0800.JPG', 'IMG_0801.JPG', 'IMG_0802.JPG', 'IMG_0803.JPG', 'IMG_0804.JPG']


In [13]:
tagged_images_with_regions = []

for key in sorted_aphanumeric(imgtagdic):
    tagid,x,y,w,h = imgtagdic[key]
    regions = [ Region(tag_id=tagid, left=x,top=y,width=w,height=h) ]

    try:
        print(key.replace(',','-'))
        print(key + " / " + allphotos[int(key.split(',')[0])])
        print("-----")
        with open(FileName+"/" + allphotos[int(key.split(',')[0])], mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=key.replace(',','-'), contents=image_contents.read(), regions=regions))
    except:
        print("Error occured")


0-1Çay
0,1Çay / IMG_0758.JPG
-----
1-1Çay
1,1Çay / IMG_0759.JPG
-----
2-1Çay
2,1Çay / IMG_0760.JPG
-----
3-1Çay
3,1Çay / IMG_0761.JPG
-----
4-1Çay
4,1Çay / IMG_0762.JPG
-----
5-1Çay
5,1Çay / IMG_0763.JPG
-----
6-1Çay
6,1Çay / IMG_0764.JPG
-----
7-1Çay
7,1Çay / IMG_0765.JPG
-----
8-1Çay
8,1Çay / IMG_0766.JPG
-----
9-1Çay
9,1Çay / IMG_0767.JPG
-----
10-1Çay
10,1Çay / IMG_0768.JPG
-----
11-1Çay
11,1Çay / IMG_0769.JPG
-----
12-1Çay
12,1Çay / IMG_0770.JPG
-----
13-1Çay
13,1Çay / IMG_0771.JPG
-----
14-1Çay
14,1Çay / IMG_0772.JPG
-----
15-1Çay
15,1Çay / IMG_0773.JPG
-----
16-1Çay
16,1Çay / IMG_0774.JPG
-----
17-1Çay
17,1Çay / IMG_0775.JPG
-----
18-1Çay
18,1Çay / IMG_0776.JPG
-----
19-1Çay
19,1Çay / IMG_0777.JPG
-----
20-1Çay
20,1Çay / IMG_0778.JPG
-----
21-1Çay
21,1Çay / IMG_0779.JPG
-----
22-1Çay
22,1Çay / IMG_0780.JPG
-----
23-1Çay
23,1Çay / IMG_0781.JPG
-----
24-1TürkKahvesi
24,1TürkKahvesi / IMG_0782.JPG
-----
25-1TürkKahvesi
25,1TürkKahvesi / IMG_0784.JPG
-----
26-1TürkKahvesi
26,1TürkKa

In [14]:
len(tagged_images_with_regions)

46

In [15]:
# For large photosets
# Upload all photos with groups of 10 photos  
for a in range(int(len(tagged_images_with_regions)/10)+1):  
        print("Progress: " + str(a) +"/"+str(int(len(tagged_images_with_regions)/10)))
        trainer.create_images_from_files(project.id, images=tagged_images_with_regions[(a*10):((a*10)+10)])
        

Progress: 0/4
Progress: 1/4
Progress: 2/4
Progress: 3/4
Progress: 4/4


In [16]:
# Training part of the project
# Just make sure everyphoto has minimum 15 photos
# If not it'll breake
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


In [17]:
# Now there is a trained endpoint that can be used to make a prediction
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

# Open the sample image and get back the prediction results.
with open("test.jpeg", mode="rb") as test_data:
    results = predictor.predict_image(project.id, test_data, iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100), prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height)

	Çay: 1.28% 0.05613403 0.005853772 0.232669011 0.219294071
	Çay: 5.25% 0.319693983 0.2955585 0.212485254 0.280546963
	Çay: 2.88% 0.06606093 0.00717167556 0.4532976 0.188685969
	Çay: 12.67% 0.0553295165 0.0613341779 0.25782758 0.48451376
	Çay: 1.34% 0.2681377 0.2407206 0.670712352 0.74453783
	Çay: 18.96% 0.00578331947 0.35564512 0.680699766 0.644344866
	TürkKahvesi: 5.02% 0.05613403 0.005853772 0.232669011 0.219294071
	TürkKahvesi: 64.56% 0.303515881 0.298589 0.217154652 0.281944633
	TürkKahvesi: 78.71% 0.05160275 0.0421375036 0.262363851 0.536808252
	TürkKahvesi: 92.06% 0.614246845 0.255478173 0.265629053 0.5214398
	TürkKahvesi: 3.46% 0.06606093 0.00717167556 0.4532976 0.188685969
	TürkKahvesi: 1.65% 0.102079511 0.01610978 0.4887963 0.477765918
	TürkKahvesi: 7.63% 0.0153126419 0.432339221 0.6618843 0.507128
